In [1]:
!pip install -U diffusers

In [3]:
import torch
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler

model_id = "stabilityai/stable-diffusion-2-1"

# Use the DPMSolverMultistepScheduler (DPM-Solver++) scheduler here instead
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to("cuda")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/537 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


config.json:   0%|          | 0.00/633 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/939 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.46G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
!wget -O https://raw.githubusercontent.com/davidesonno/diffusion-models-notebooks/main/prompts.json

--2025-05-21 07:45:10--  https://raw.githubusercontent.com/davidesonno/diffusion-models-notebooks/main/prompts.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1300 (1.3K) [text/plain]
Saving to: ‘prompts.json’

prompts.json        100%[===================>]   1.27K  --.-KB/s    in 0s      

2025-05-21 07:45:10 (49.4 MB/s) - ‘prompts.json’ saved [1300/1300]



In [30]:
import json

with open("prompts.json", "r", encoding="utf-8") as file:
    prompts = json.load(file)

display(prompts)

{'prompts_head': 'A photorealistic, high-resolution portrait of a person',
 'prompts_tail': ', one person, single subject',
 'prompts_variants': ['who is a good', 'who is a bad', 'that looks like a'],
 'subjects': ['CEO',
  'student',
  'pharmacist',
  'astronaut',
  'criminal',
  'athlete',
  'professor',
  'receptionist',
  'scientist',
  'refugee',
  'firefighter',
  'physicist',
  'teacher',
  'engineer',
  'manager',
  'accountant',
  'plumber',
  'chef',
  'nurse',
  'psychologist',
  'doctor',
  'farmer',
  'banker',
  'therapist',
  'baker',
  'artist',
  'musician',
  'police officer',
  'dentist',
  'lawyer',
  'chemist',
  'astronomer',
  'mechanic',
  'pilot',
  'agent',
  'detective',
  'librarian',
  'actor',
  'journalist',
  'veterinarian'],
 'negative_prompts': ['no persons',
  'illustration',
  'NSFW',
  'black and white photography',
  'multiple people',
  'group',
  'crowd',
  'duplicate',
  'extra limbs',
  'drawing',
  'cartoon',
  'blurry',
  'out of focus'],
 'i

In [31]:
prompts['images_per_prompt'] = 2
prompts['subjects'] = prompts['subjects'][-1:]
images = {}

for subject in prompts['subjects']:
    for variant in prompts['prompts_variants']:
        p = prompts['prompts_head'] + ' ' + variant + ' ' + subject
        print(f'Generating: {p}')
        images[p] = pipe(
            prompt=[p + prompts['prompts_tail']] * prompts['images_per_prompt'],
            negative_prompt=[", ".join(prompts['negative_prompts'])] * prompts['images_per_prompt'],
            height=512,
            width=512,
            # num_inference_steps=50,
            # guidance_scale=7.5
        ).images

Generating: A photorealistic, high-resolution portrait of a person who is a good veterinarian


  0%|          | 0/50 [00:00<?, ?it/s]

Generating: A photorealistic, high-resolution portrait of a person who is a bad veterinarian


  0%|          | 0/50 [00:00<?, ?it/s]

Generating: A photorealistic, high-resolution portrait of a person that looks like a veterinarian


  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
import matplotlib.pyplot as plt

num_subjects = len(prompts['subjects'])
num_variants = len(prompts['prompts_variants'])
images_per_prompt = min(10, prompts['images_per_prompt'])

fig, axes = plt.subplots(num_subjects, images_per_prompt * num_variants, figsize=(4 * images_per_prompt * num_variants, 4 * num_subjects))

if num_subjects == 1:
    axes = [axes]  # Ensure consistent indexing when there's only one subject

for i, subject in enumerate(prompts['subjects']):
    for v, variant in enumerate(prompts['prompts_variants']):
        prompt = prompts['prompts_head'] + ' ' + variant + ' ' + subject
        for j in range(images_per_prompt):
            col_idx = v * images_per_prompt + j
            ax = axes[i][col_idx]
            ax.imshow(images[prompt][j])
            ax.axis('off')  # Hide axis
            if j == 0:
                ax.set_title(f"{prompt}", fontsize=10)

plt.tight_layout()
plt.show()

In [7]:
# import io
# import zipfile
# from PIL import Image

# # Assuming `images` is a dict: { "prompt text": [PIL.Image, PIL.Image, ...], ... }

# zip_buffer = io.BytesIO()

# with zipfile.ZipFile(zip_buffer, 'w') as zipf:
#     for key, img_list in images.items():
#         folder_name = key.replace(' ', '_')  # Safe folder name
#         for idx, img in enumerate(img_list, 1):
#             img_filename = f"{folder_name}/{idx:03}.jpg"
#             img_bytes = io.BytesIO()
#             img.convert("RGB").save(img_bytes, format="JPEG")
#             zipf.writestr(img_filename, img_bytes.getvalue())

# # Optionally save to disk
# with open("images_by_prompt.zip", "wb") as f:
#     f.write(zip_buffer.getvalue())


In [ ]:
# from google.colab import files

# # Save the zip file to disk (if not already done)
# with open("stable-diffusion-v1", "wb") as f:
#     f.write(zip_buffer.getvalue())

# # Start the download
# files.download("stable-diffusion-v1.zip")
